<a href="https://colab.research.google.com/github/pipanolis/Tamura-job-haw-kadai20200514/blob/master/Simple_FaultDetection_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
#@title Default title text
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, precision_recall_fscore_support
from sklearn.utils.multiclass import unique_labels
from keras.layers.core import Dense, Activation, Dropout
import time #helper libraries
from tqdm import tqdm_notebook as tqdm
import sys
import os
import tensorflow_datasets as tfds

## Useful Codes

In [ ]:
cwd = os.getcwd()
cwd

'/content'

In [ ]:
# Time couonting
start = time.time()
end = time.time()
print("Time: {} seconds".format(end-start))

Time: 4.1484832763671875e-05 seconds


In [ ]:
# Each element is a dictionary mapping strings to `tf.Tensor` objects.
elements =  ([{"a": 1, "b": "foo"},
              {"a": 2, "b": "bar"},
              {"a": 3, "b": "baz"}])
dataset = tf.data.Dataset.from_generator(
    lambda: elements, {"a": tf.int32, "b": tf.string}) # lambda is the generator 

dataset.element_spec

for elem in dataset:
  print(elem)

for elem in dataset:
  print(elem['a'])


# `map_func` takes a single argument of type `dict` with the same keys
# as the elements.
result = dataset.map(lambda d: str(d["a"]) + d["b"])
for elem in result:
  print(elem)

InternalError: ignored

In [ ]:
# list(dataset.as_numpy_iterator())

# Code for testing

In [2]:
tf.keras.backend.clear_session()

DEBUG = False   # Set to True for debugging mode. Changing this value requires the runtime to restart
RELOAD = False  # Set to true of the raw files should be reloaded from disk. False will use a saved .pkl file 
VAL = False     # If validation set should be run during the training session
NAME = "lstm_all_with_stardiv" # Name of the settings/run

# A list of features to be exluded. Possible values (or None): 
# ['kal', 'kalq', 'kaldiv', 'kalref', star', 'star1', 'star2', wheel', 'mt', 'mt1', 'mt2', 'omega', 'gps', 'gpospos'
#   , 'gpsvel', 'current', 'wheel']

# A list of features 
# ['quo_error':4, 'ome_error':3]

EXCLUDE_FEATURES = []

# Number of timesteps in the sequence
SEQ_LEN = 32

# Which classes to include
INCLUDE = np.arange(1,4)
INCLUDE = np.insert(INCLUDE,0,0) 

# Number of faults/classes
NUM_CLASSES = 4

# Number of units in the LSTM
NUM_UNITS = [512, 512]

# The batch size
BATCH_SIZE = 1024

# Number of training epochs
NUM_EPOCHS = 5

# The learning rate
LR = 0.001

# Index of the first feature column
START = 2

settings =  {'EXCLUDE_FEATURES': EXCLUDE_FEATURES,
             'SEQ_LEN': SEQ_LEN,
             'INCLUDE': INCLUDE,
             'NUM_CLASSES': NUM_CLASSES,
             'NUM_UNITS': NUM_UNITS,
             'BATCH_SIZE': BATCH_SIZE,
             'NUM_EPOCHS': NUM_EPOCHS,
             'LR': LR,
             'START': START}

NUM_FEATURES = 7

In [3]:
print("{}".format(tf.executing_eagerly()))

True


# Load Data

In [4]:
train_file = "/content/drive/MyDrive/Simulation_data/220117_1432_20itrs.csv"
test_file  = "/content/drive/MyDrive/Simulation_data/220117_1409_10itrs.csv"


# load the dataset
df_train = read_csv(train_file, header=None, index_col=None, delimiter='  ')
df_test  = read_csv(test_file, header=None, index_col=None, delimiter='  ')
np_df_train = df_train.values
np_df_test  = df_test.values

n_features = df_train.columns
# print(df)
print(np_df_train[0:10])
print(n_features)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e-02 4.5140619e-04 1.9997973e-03 4.4498909e-04
  3.2452102e-06 9.0562445e-02 3.9991982e-01 8.7994518e-02]
 [0.0000000e+00 2.0000000e-02 9.0556751e-04 3.9991864e-03 8.7991323e-04
  1.2971720e-05 9.1113377e-02 3.9984082e-01 8.5978224e-02]
 [0.0000000e+00 3.0000000e-02 1.3623976e-03 5.9981630e-03 1.3047089e-03
  2.9165942e-05 9.1652741e-02 3.9976306e-01 8.3951347e-02]
 [0.0000000e+00 4.0000000e-02 1.8218097e-03 7.9967232e-03 1.7193143e-03
  5.1814415e-05 9.2180482e-02 3.9968656e-01 8.1914118e-02]
 [0.0000000e+00 5.0000000e-02 2.2837162e-03 9.9948632e-03 2.1236690e-03
  8.0903817e-05 9.2696547e-02 3.9961136e-01 7.9866767e-02]
 [0.0000000e+00 6.0000000e-02 2.7480293e-03 1.1992579e-02 2.5177143e-03
  1.1642097e-04 9.3200884e-02 3.9953749e-01 7.7809526e-02]
 [0.0000000e+00 7.0000000e-02 3.2146604e-03 1.3989868e-02 2.9013933e-03
  1.5835285

# Scale Data

In [5]:
def scale(data):

  data[:,2:6] /= 1 # quaternion error
  data[:,6:9] /= 1 # angular velocity error 

  return data


data_train = scale(np_df_train)
data_test = scale(np_df_test)

# Valid Indices function def

In [6]:
def valid_indices(data, seq_len=32, undersample=True, include_list=np.arange(35), n_classes=35):

  fault_dict = {0: 'Nominal', 1: 'Gyro Freeze', 2: 'Gyro Bias', 3: 'STT Freeze',
                4: 'Reduced Mg Mmt', 5: 'Residual Mg mmt'}
    
  indices = np.arange(0, data.shape[0]-seq_len)
  labels = {idx: [] for idx in indices} # An empty dictionary to store the future classes/labels

  idxs_del = []
  for idx in tqdm(indices):
    # Check for class discontinuity
    if data[idx+seq_len][1] < data[idx][1]: 
      # Invalid sample, append to deletion index
      idxs_del.append(idx)
    else:
      # # Valid sample, determine the label
      # if 'MA' not in fault_dict[data[idx][1]]: # Check if it is an MA fault
      #   # Non MA Fault, check if t_n > fault_start
      #   labels[idx] = data[idx][1] if (data[idx+seq_len][0] > data[idx][2]) else 0
      # else:
      #   # MA fault, no need to check time
      labels[idx] = int(data[idx+seq_len][0])
  # Delete the unwanted indices
  [labels.pop(e) for e in idxs_del]
  labels_old = labels

  return labels, indices

# Call the function
labels, indices = valid_indices(data_train, seq_len=SEQ_LEN, undersample=True, include_list=INCLUDE, n_classes=NUM_CLASSES)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/99988 [00:00<?, ?it/s]

In [7]:
print(labels.values())
print(len(labels.values()))

dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Input function def

In [8]:
# def input_fn(seq_len=32, batch_size=1024, num_classes=35, num_epochs=10, 
#              dtype='train', shuffle=True, start=3, num_features=50):

  # # Load data
  # with tf.io.gfile.GFile('/content/{0}.npy'.format(dtype), 'rb') as f:
  #   data = np.load(f)
  # # Load indices
  # with tf.io.gfile.GFile('/content/{0}_indices.npy'.format(dtype), 'rb') as f:
  #   indices = np.load(f)
  # # Load labels
  # with tf.io.gfile.GFile('/content/{0}_labels.pkl'.format(dtype), 'rb') as f:
  #   label_dict = pickle.load(f)

# @tf.autograph.experimental.do_not_convert
def input_fn(labels, indices, data, seq_len=32, batch_size=1024, num_classes=35, num_epochs=10, 
             dtype='train', shuffle=True, start=3, num_features=50):

  label_values = [x for x in labels.values()]

  # if shuffle:
  #   arr = np.arange(len(indices))
  #   np.random.shuffle(arr)
  #   indices = indices[arr]
  #   label_values = np.array(label_values)[arr]

  # Create constants  
  label_values = tf.constant(label_values, dtype=tf.int32)
  indices = tf.constant(indices, dtype=tf.int32)
  data = tf.constant(data, dtype=tf.float32)

  print(data)

  # Construct dataset
  ds = tf.data.Dataset.from_tensor_slices(indices)

  # Map dataset as sequence of length seq_len and labels
  ds1 = ds.map(lambda x: tf.slice(data, [x, start], [seq_len, num_features]))
  ds2 = tf.data.Dataset.from_tensor_slices(label_values)

  ds3 = tf.data.Dataset.zip((ds1,ds2.map(lambda x: tf.one_hot(tf.cast(x, dtype=tf.int32), depth=num_classes)),ds))
  ds3 = ds3.repeat(num_epochs).batch(batch_size, drop_remainder=True)
  return ds3.map(lambda a,b,c: (a,b)), ds3.map(lambda a,b,c: c)

# Call the function
newdata_labels, newindices = input_fn(labels, indices, data_train, seq_len=SEQ_LEN, batch_size=BATCH_SIZE, num_classes=NUM_CLASSES, 
                                      num_epochs=NUM_EPOCHS, dtype='train', shuffle=True, start=2, num_features=7)

tf.Tensor(
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 9.9999998e-03 4.5140620e-04 ... 9.0562448e-02
  3.9991981e-01 8.7994516e-02]
 [0.0000000e+00 2.0000000e-02 9.0556749e-04 ... 9.1113374e-02
  3.9984083e-01 8.5978225e-02]
 ...
 [3.0000000e+00 4.9980000e+01 4.5811487e-03 ... 3.9769942e-03
  1.7673793e-03 1.6445007e-02]
 [3.0000000e+00 4.9990002e+01 4.9277204e-03 ... 4.3861857e-03
  1.7052577e-03 1.8572694e-02]
 [3.0000000e+00 5.0000000e+01 5.7959375e-03 ... 3.2269568e-04
  2.3202321e-03 2.6078764e-04]], shape=(100020, 9), dtype=float32)


In [9]:
# list(ds1.as_numpy_iterator())

LSTM Model

In [10]:
def lstm_model(seq_len=32, batch_size=None, stateful=True, n_features=10, num_classes=35, 
              num_units=[32, 32]):
  source = tf.keras.Input(
  name='seed', shape=(seq_len, n_features), 
      batch_size=batch_size)
  
  lstm_1 = tf.keras.layers.LSTM(num_units[0], stateful=stateful, return_sequences=True,dropout=0.1, recurrent_dropout=0.1)(source)
  lstm_2 = tf.keras.layers.LSTM(num_units[1], stateful=stateful, return_sequences=False, dropout=0.1)(lstm_1)
  #dense_1 = tf.keras.layers.Dense(100, activation='relu')(lstm_2)
  
  predict = tf.keras.layers.Dense(num_classes, activation='softmax')(lstm_2)
  
  return tf.keras.Model(inputs=[source], outputs=[predict])


# Training
## Training with a normal machine

In [11]:
# tf.keras.backend.clear_session()
# model = lstm_model(seq_len=SEQ_LEN, batch_size=BATCH_SIZE, num_units=NUM_UNITS, n_features=7, num_classes=NUM_CLASSES)
# print(model.summary())
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# history = model.fit(newdata_labels, epochs=NUM_EPOCHS, steps_per_epoch=train_steps)

## Training with TPU

In [14]:
tf.keras.backend.clear_session()
ds, idxs = input_fn(labels, indices, data_train, seq_len=SEQ_LEN, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS,
                    num_classes=NUM_CLASSES, start=START, num_features=NUM_FEATURES,
                    dtype='train', shuffle=True)

# Setting up the TPU Environment
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

tf.Tensor(
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 9.9999998e-03 4.5140620e-04 ... 9.0562448e-02
  3.9991981e-01 8.7994516e-02]
 [0.0000000e+00 2.0000000e-02 9.0556749e-04 ... 9.1113374e-02
  3.9984083e-01 8.5978225e-02]
 ...
 [3.0000000e+00 4.9980000e+01 4.5811487e-03 ... 3.9769942e-03
  1.7673793e-03 1.6445007e-02]
 [3.0000000e+00 4.9990002e+01 4.9277204e-03 ... 4.3861857e-03
  1.7052577e-03 1.8572694e-02]
 [3.0000000e+00 5.0000000e+01 5.7959375e-03 ... 3.2269568e-04
  2.3202321e-03 2.6078764e-04]], shape=(100020, 9), dtype=float32)
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.99.23.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.99.23.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [15]:
# Creating and compiling the model
with strategy.scope():
  training_model = lstm_model(seq_len=SEQ_LEN, stateful=False, num_units=NUM_UNITS, n_features=NUM_FEATURES,
                             num_classes=NUM_CLASSES, batch_size=BATCH_SIZE)
  # training_model = conv1D_model(seq_len=SEQ_LEN, n_features=NUM_FEATURES, num_classes=NUM_CLASSES)
  adam= tf.keras.optimizers.Adam(learning_rate=LR)
  # sgd = tf.keras.optimizers.SGD(momentum=0.006)
  training_model.compile(
      optimizer=adam,
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy']) # Compile with adam optimizer


NUM_EPOCHS = 20
history = training_model.fit(ds, 
                             epochs=NUM_EPOCHS, 
                            #  steps_per_epoch=train_steps,
                            # callbacks=[checkpoint], 
                            # validation_data=ds_test if VAL else None, 
                            # validation_steps=test_steps if VAL else None
                             )



Epoch 1/20
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


1941/1941 [==============================] - 63s 29ms/step - loss: 1.2750 - categorical_accuracy: 0.4854
Epoch 2/20
1596/1941 [=======================>......] - ETA: 10s - loss: 1.1757 - categorical_accuracy: 0.5140

KeyboardInterrupt: ignored

In [ ]:
# %matplotlib inline
fig, ax = plt.subplots()
plt.plot(np.arange(1,NUM_EPOCHS+1),history.history['categorical_accuracy'],'b-', linewidth=3, markersize=10, label='Training')

fig.set_size_inches(5,5)
ymax = max(history.history['categorical_accuracy'])
xmax = history.history['categorical_accuracy'].index(ymax)
plt.annotate('Max training accuracy = {0:.3g}'.format(ymax), xy=(xmax, ymax), 
             xytext=(4, 0.82), fontsize=15)

plt.legend(loc=7, borderaxespad=1, fontsize=12)
plt.title('Training accuracy over epochs', fontsize=20)
ax.set_xlabel('Epoch', fontsize=20)
ax.set_ylabel('Categorical Accuracy', fontsize=20)
plt.xticks(np.arange(0,NUM_EPOCHS+2,2), fontsize=15)
plt.yticks(fontsize=15)


In [ ]:
from time import mktime, gmtime, strftime
t = strftime("%y%m%d_%H:%M:%S", gmtime(mktime(gmtime()) + 9*3600))
fig.savefig('/content/drive/MyDrive/DL_result/{}categorical_accuracy.png'.format(t))


NAME = 'trained_model'
# training_model.save('/content/lstm.hdf5', overwrite=True)
training_model.save('/content/drive/MyDrive/DL_result/{0}_{1}.hdf5'.format(NAME,t), overwrite=True)
training_model.save_weights('/content/drive/MyDrive/DL_result/{0}_{1}_weights.h5'.format(NAME,t), overwrite=True)

Confusion matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.YlGn, 
                          annotate=True,
                          fig_size=10):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = [classes[x] for x in unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap, alpha=1.0)
    ax.figure.colorbar(im, ax=ax )

    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    fig.tight_layout()
    
    # Lines and size
    for pos in np.arange(0.5, len(classes)+0.5):
      ax.axhline(pos, linestyle='--', c='green')
      ax.axvline(pos, linestyle='--', c='green')
    
    fig.set_size_inches(fig_size,fig_size)
    
    if annotate:
      # Loop over data dimensions and create text annotations.
      fmt = '.2f' if normalize else 'd'
      thresh = cm.max() / 2.
      for i in range(cm.shape[0]):
          for j in range(cm.shape[1]):
              ax.text(j, i, format(cm[i, j], fmt),
                      ha="center", va="center",
                      color="white" if cm[i, j] > thresh else "black",
                     size=fig_size*1.5)
    fig.tight_layout()
    return ax

In [ ]:
# fault_dict = {0: 'Nominal', 1: 'Gyro Freeze', 2: 'Gyro Bias', 3: 'STT Freeze'}


# ## TRAIN SET
# # with open('/content/test_labels.pkl', 'rb') as f:
# #   labels = pickle.load(f)

# CLASSES = [x for x in fault_dict.values()]
# y_true = [int(x) for x in labels.values()]
# y_pred = np.argmax(predictions_test, axis=1)
# true_class = [CLASSES[x] for x in y_true]
# pred_class = [CLASSES[x] for x in y_pred]

# plot_confusion_matrix(y_true[0:len(y_pred)], 
#                       y_pred, 
#                       classes = CLASSES, 
#                       normalize=True, 
#                       title="Confusion Matrix for training data")

# plt.savefig('{0}_confusion_test.pdf'.format(NAME))

#Prediction Test

In [ ]:
BATCH_SIZE = 1024
file_name = '/content/drive/MyDrive/DL_result/{0}_{1}.hdf5'.format(NAME,t)

print(file_name)

labels, indices = valid_indices(data_test, seq_len=SEQ_LEN)
# indices = indices[0:92940]
# test_steps = int(np.floor(len(indices)/BATCH_SIZE)) # Number of steps in the dataset

# # Save the testing indices to file
# with open('/content/test_indices.npy', 'wb') as f:
#   np.save(f, indices)

# with open("test_labels.pkl", 'wb') as f:
#   pickle.dump(labels,f)

ds_test, _ = input_fn(labels, indices, data_test, seq_len=SEQ_LEN, batch_size=BATCH_SIZE, num_epochs=1, 
                      num_classes=NUM_CLASSES, start=START, num_features=NUM_FEATURES,
                      dtype='test', shuffle=False)

prediction_model = lstm_model(seq_len=SEQ_LEN, batch_size=BATCH_SIZE, stateful=False, n_features=NUM_FEATURES, num_classes=NUM_CLASSES, num_units=NUM_UNITS)
prediction_model.compile(optimizer=tf.keras.optimizers.Adam(lr=LR), 
                         loss='categorical_crossentropy', 
                         metrics=['categorical_accuracy']) # Compile with adam optimizer

                         
prediction_model.load_weights(file_name)
prediction_model.reset_states()
tf.keras.utils.plot_model(prediction_model, show_shapes=True, to_file='{0}_architecture.png'.format(NAME))
print("Starting predictions...")
predictions_test = prediction_model.predict(ds_test, 
                                            # steps=test_steps,
                                            verbose=1)

Confusion Matrix

In [ ]:
fault_dict = {0: 'Nominal', 1: 'Gyro Freeze', 2: 'Gyro Bias', 3: 'STT Freeze',
              4: 'Reduced Mg Mmt', 5: 'Residual Mg mmt'}


## TESTING SET
# with open('/content/test_labels.pkl', 'rb') as f:
#   labels = pickle.load(f)

CLASSES = [x for x in fault_dict.values()]
y_true = [int(x) for x in labels.values()]
y_pred = np.argmax(predictions_test, axis=1)
true_class = [CLASSES[x] for x in y_true]
pred_class = [CLASSES[x] for x in y_pred]

plot_confusion_matrix(y_true[0:len(y_pred)], 
                      y_pred, 
                      classes = CLASSES, 
                      normalize=True, 
                      title="Confusion Matrix for testing data",
                      fig_size=7)

plt.savefig('/content/drive/MyDrive/DL_result/{0}_confusion_test.png'.format(NAME))